In [3]:
#___________________________________________________________________________________________________________________
# get_ipython().magic('matplotlib notebook')
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import pyvista as pv
import vtk
import tripyview as tpv

In [4]:
# Parameters
mesh_path ='/work/ollie/projects/clidyn/FESOM2/meshes/core2/'
save_path = None #'~/figures/test_papermill/'
save_fname= None

#_____________________________________________________________________________________
which_cycl= 3
which_mode= '3dsphere'

#_____________________________________________________________________________________
input_paths= list()
input_paths.append(None)

input_names= list()
input_names.append('resolution')

vname     = 'elevation'
year      = [1979,2019]
mon, day, record, box, depth = None, None, None, None, 100

#_____________________________________________________________________________________
# do anomaly plots in case ref_path is not None
ref_path  = None #'/home/ollie/pscholz/results/trr181_tke_ctrl_ck0.1/' # None
ref_name  = None #'TKE, ck=0.1' # None
ref_year  = None # [2009,2019]
ref_mon, ref_day, ref_record = None, None, None

#_____________________________________________________________________________________
do_clim   = False
which_clim= 'phc3'
clim_path = '/work/ollie/pscholz/INIT_HYDRO/phc3.0/phc3.0_annual.nc'

#_____________________________________________________________________________________
cmap      = 'rygbw'
cnum      = 20
cref      = None
crange, cmin, cmax, cfac = None, None, None, None

#_____________________________________________________________________________________
ncolumn       = 1 
do_rescale    = False
which_dpi     = 300
do_potatoefac = 0.6
do_topo       = True


In [5]:
#___LOAD FESOM2 MESH___________________________________________________________________________________
mesh=tpv.load_mesh_fesom2(mesh_path, do_rot='None', focus=0, do_info=True, do_pickle=True,
                      do_earea=True, do_narea=True, do_eresol=[True,'mean'], do_nresol=[True,'eresol'])

#______________________________________________________________________________________________________
if which_cycl is not None: 
    for ii,ipath in enumerate(input_paths):
        if ipath is not None: 
            input_paths[ii] = os.path.join(ipath,'{:d}/'.format(which_cycl))
            print(ii, input_paths[ii])
    
    if ref_path is not None:
        ref_path = os.path.join(ref_path,'{:d}/'.format(which_cycl))
        print('R', ref_path)
#______________________________________________________________________________________________________
cinfo=dict({'cstr':cmap, 'cnum':cnum})     
if crange   is not None: cinfo['crange']=crange
if cmin     is not None: cinfo['cmin'  ]=cmin
if cmax     is not None: cinfo['cmax'  ]=cmax
if cref     is not None: cinfo['cref'  ]=cref
if cfac     is not None: cinfo['cfac'  ]=cfac
if ref_path is not None: cinfo['cref'  ]=0.0    
    
#______________________________________________________________________________________________________
# in case of diff plots
if ref_path is not None:
    if ref_year   is None: ref_year   = year
    if ref_mon    is None: ref_mon    = mon
    if ref_record is None: ref_record = record

 > found *.pckl file: /work/ollie/pscholz/meshcache_tripyview/core2
 > load  *.pckl file: tripyview_fesom2_core2.pckl
___FESOM2 MESH INFO________________________
 > path            = /work/ollie/projects/clidyn/FESOM2/meshes/core2
 > id              = core2
 > do rot          = None
 > [al,be,ga]      = 50, 15, -90
 > do augmpbnd     = True
 > do cavity       = False
 > do lsmask       = True
 > do earea,eresol = True, True
 > do narea,nresol = True, True
___________________________________________
 > #node           = 126858
 > #elem           = 244659
 > #lvls           = 48
___________________________________________


In [6]:
# #___LOAD FESOM2 REFERENCE DATA________________________________________________________________________
# if ref_path is not None:
#     if depth =='bottom':
#         data_ref = tpv.load_data_fesom2(mesh, ref_path, vname=vname, year=ref_year, mon=ref_mon, day=ref_day, record=ref_record, 
#                                       depth=None, descript=ref_name, do_info=False) 
#         data_ref = data_ref.isel(nz=xr.DataArray(mesh.n_iz, dims='nod2'), nod2=xr.DataArray(range(0,mesh.n2dn), dims='nod2'))
#     else:
#         data_ref = tpv.load_data_fesom2(mesh, ref_path, vname=vname, year=ref_year, mon=ref_mon, day=ref_day, record=ref_record, 
#                                       depth=depth, descript=ref_name, do_info=False) 

# #___LOAD FESOM2 DATA___________________________________________________________________________________
# data_list = list()
# for datapath,descript in zip(input_paths, input_names): 
#     print(datapath)
#     if depth =='bottom':
#         data = tpv.load_data_fesom2(mesh, datapath, vname=vname, year=year, mon=mon, day=day, record=record, 
#                                       depth=None, descript=descript, do_info=False) 
#         data = data.isel(nz=xr.DataArray(mesh.n_iz, dims='nod2'), nod2=xr.DataArray(range(0,mesh.n2dn), dims='nod2'))
#     else:
#         data = tpv.load_data_fesom2(mesh, datapath, vname=vname, year=year, mon=mon, day=day, record=record, 
#                                       depth=depth, descript=descript, do_info=False) 
#     #__________________________________________________________________________________________________    
#     if ref_path is not None:
#         data_list.append(do_anomaly(data, data_ref))  
#     else:
#         data_list.append(data)  
#     del(data)
    
# if ref_path is not None: del(data_ref)    
    
# #___APPEND ABS CLIMATOLOGY_____________________________________________________________________________    
# if (vname in ['temp', 'salt', 'pdens'] or 'sigma' in vname) and (depth is not 'bottom') and do_clim and (ref_path is None): 
#     clim_vname = vname
#     if   vname=='temp' and  which_clim.lower()=='woa18': clim_vname = 't00an1'
#     elif vname=='salt' and  which_clim.lower()=='woa18': clim_vname = 's00an1'
#     clim = tpv.load_climatology(mesh, clim_path, clim_vname, depth=depth)
#     data_list.append(clim)    
#     del(clim)

In [7]:
print(mesh.n_z.min())
print(mesh.n_z.max())
# print(data_list[0])
# fig=plt.figure()
# for lsmask in mesh.lsmask_a:
#     plt.plot(lsmask[:,0],lsmask[:,1])
# plt.plot(mesh.n_x[mesh.n_i==1], mesh.n_y[mesh.n_i==1],'*')

-6000.0
-30.0


In [8]:
# #
# #
# #___________________________________________________________________________________________________________________
# class widget_lon_lat_zoom():
#     def __init__(self, plt, init_lon, init_lat, init_zoom):
#         # default parameters
#         self.kwargs = {'center_lon': init_lon, 'center_lat': init_lat, 'zoom_fac': init_zoom}
#         x,y,z = tpv.grid_cart3d(self.kwargs['center_lon'], self.kwargs['center_lat'], R_earth*10/self.kwargs['zoom_fac'],is_deg=True)
#         plt.camera_position = [ np.array([x,y,z]),(0,0,0),(0,0,1)]   

#     def __call__(self, param, value):
#         self.kwargs[param] = value
#         self.update()

#     def update(self):
#         x,y,z = tpv.grid_cart3d(self.kwargs['center_lon'],self.kwargs['center_lat'], R_earth*10/self.kwargs['zoom_fac'], is_deg=True)
#         plt.camera_position = [ np.array([x,y,z]),(0,0,0),(0,0,1)]    
#         return

In [9]:
do_var        = 'elevation'
# do_var        = 'resolution'
do_potatoefac = 0.6
do_topo       = True

#
#
#___CREATE PYVISTA OCEAN MESH___________________________________________________________
meshpv_ocean = tpv.create_3d_ocean_mesh(mesh,potatoefac=do_potatoefac, variable=vname)

#
#
#___CREATE PYVISTA LAND MESH TO FILL HOLES______________________________________________
# # meshpv_land = create_3d_land_mesh(mesh)
meshpv_land = tpv.create_3d_land_mesh(mesh,
                                  potatoefac  = 0.25,
                                  resol       = 0.3,
                                  do_topo     = do_topo, 
                                  topo_path   = '/work/ollie/pscholz/bathmetry/topo_ETOPO5/topo_1deg.nc',
                                  topo_varname= 'topo',
                                  topo_dimname= ['lon','lat'])
#
#
#___CREATE PYVISTA 3d COASTLINE_________________________________________________________
points_coast = tpv.create_3d_coastline(mesh)    

#
#
#___CREATE PYVISTA 3D LONLAT GRID_______________________________________________________
points_lonlat_grid = tpv.create_3d_lonlat_grid(dlon=30,dlat=15,potatoefac=0.3,do_topo=do_topo)


 --> compute 3d ocean mesh
 --> compute 3d land mesh
 --> compute 3d coastline
 --> compute 3d longitude grid
 --> compute 3d latitude grid


In [10]:
def close_window(plt):
    plt.close()

In [11]:
#___CREATE PYVISTA OCEAN MESH___________________________________________________________
# do_mode    = 'interactive' # 'interactive', 'movie', 'print'
# do_mode    = 'movie' # 'interactive', 'movie', 'print'
do_mode    = 'print' # 'interactive', 'movie', 'print'
do_wsize   = [5000,5000] #determines resolution when printing
do_contour = False
do_smooth  = True
init_lon,init_lat,init_zoom = -30, 50, 2


#_______________________________________________________________________________________
# create custom colormap
if vname=='elevation':
    cmin, cmax, cref = 0,6000,3000
    str_title = 'Depth [m]'
    which_cmap = 'wbgyr'
    cmap,clevel = tpv.colormap_c2c(cmin,cmax,cref,20,which_cmap)
    
elif  vname=='resolution':    
    cmin = np.floor(meshpv_ocean['resolution'].min())
    print(cmin)
    cmax = np.ceil(meshpv_ocean['resolution'].max())
    print(cmax)
    cref = cmin + (cmax-cmin)/2
    cref = np.around(cref, -np.int32(np.floor(np.log10(np.abs(cref)))-1+1)) 
    print(cref)
    cstep_list=np.arange(0,250+1,5)
    cstep_idx = np.argmin(np.abs(cstep_list-cmin))
    cmin = cstep_list[cstep_idx]
    cstep_idx = np.argmin(np.abs(cstep_list-cmax))
    cmax = cstep_list[cstep_idx]
    str_title = 'Resolution \n [km]'
    which_cmap = 'rygbw'
    print(cmin,cmax,cref, (cmax-cmin)/5,which_cmap)     
    cmap,clevel = tpv.colormap_c2c(cmin,cmax,cref,(cmax-cmin)/5,which_cmap)
print(cmin,cmax,cref)
print(clevel.shape)
clim = [clevel[0],clevel[-1]]

#_______________________________________________________________________________________
# create pyvista plot object
# pv.set_plot_theme("default")
pv.set_plot_theme("document")
# pv.set_plot_theme("dark")
# pv.set_plot_theme("night")
# pv.set_plot_theme("ParaView")

if do_mode=='print':
    pv.set_plot_theme("document")
    plt = pv.Plotter(notebook=True, off_screen=True, window_size=[900,900]) #, multi_samples=4, line_smoothing=False, polygon_smoothing=False)
    plt.background_color=None
elif do_mode=='movie':
    plt = pv.Plotter(notebook=False,window_size=[1000,1000])    
else:
    plt = pv.Plotter(notebook=False)    

    
#_______________________________________________________________________________________
# colorbar arguments
fsize_label, fsize_title = 12, 24
if do_mode =='print': fsize_label, fsize_title = fsize_label*6, fsize_title*4
dict_sargs = dict(vertical=True, 
             height=0.75, 
             position_x=0.875, 
             shadow=True,
             position_y=0.125,
             title_font_size=fsize_title,
             label_font_size=fsize_label,
             fmt="%4.0f",
             n_labels=np.int8(len(clevel)),#n_labels=np.int8(len(clevel)/2)+1,
             n_colors=cmap.N,
             outline=False)

dict_annot=dict()

# _______________________________________________________________________________________
# plot 3d ocean mesh
print('--> draw ocean mesh')
plt.add_mesh(meshpv_ocean, 
             clim=clim,
             cmap=cmap, 
             scalars=do_var,
             show_edges=False, 
             edge_color='black',
             show_scalar_bar=True,
             scalar_bar_args=dict_sargs,
             diffuse=1.0, specular=0.1, specular_power=2, ambient=0.1,
             lighting=True,#             stitle=str_title,
             n_colors=cmap.N,
             annotations=dict_annot,
             smooth_shading=do_smooth)

if do_contour:
    contours = meshpv_ocean.contour(scalars='elevation', isosurfaces=np.array([250,500,1000,1500,2000,3000,4000,5000,6000]))
    plt.add_mesh(contours, color="black", line_width=1,smooth_shading=True)
    
#_______________________________________________________________________________________
# plot 3d land mesh
print('--> draw land mesh')
plt.add_mesh(meshpv_land,color=[0.7,0.7,0.7],
             diffuse=1.0, specular=0.1, specular_power=2, ambient=0.1,
             lighting=True,
             smooth_shading=do_smooth)

#_______________________________________________________________________________________
# plot 3d coast, lon-grid and lat-grid lines
print('--> draw coast lines')
for points in points_coast:
    plt.add_lines(points,      color='black',width=1)
print('--> draw grid lines ')    
for points in points_lonlat_grid:    
    plt.add_lines(points,      color='black',width=1)


#_______________________________________________________________________________________
# do slider widget infrastructure
if do_mode == 'interactive':
    wgap    = 0.05 
    wslider = (1-4*wgap)/3
    engine  = tpv.widget_lon_lat_zoom(plt, init_lon,init_lat,init_zoom)
    plt.add_slider_widget(lambda value: engine(plt, 'center_lon', int(value)), 
                          [-180, 180], value = int(init_lon), title='lon', style='modern',      
                          pointa=(1*wgap+0*wslider, .06), pointb=(1*wgap+1*wslider, .06))
    plt.add_slider_widget(lambda value: engine(plt, 'center_lat', int(value)),
                          [ -90,  90], value =  int(init_lat), title='lat', style='modern' ,  
                          pointa=(2*wgap+wslider, .06),   pointb=(2*wgap+2*wslider, .06))
    plt.add_slider_widget(lambda value: engine(plt, 'zoom_fac',   value),
                          [0.1,  10],   value =  init_zoom,  title='zoom_fac', style='modern', 
                          pointa=(3*wgap+2*wslider, .06), pointb=(3*wgap+3*wslider, .06))
    
#     plt.add_key_event('q', close_window(plt))
    plt.show()
#     plt.show(auto_close=False, interactive_update=False)
#     plt.close()

#_______________________________________________________________________________________
# do animated mp4 of rotating earth 
elif do_mode =='movie':
    
    frame_nr    = 720
    frame_rate  = 16
    frame_wsize = [1008,1008]
#     frame_wsize = [800,800]
    fname       = 'movie_rotate_3dglobe_'+do_var+'.mp4'
    plt.open_movie(fname, framerate=frame_rate)
    
    #___________________________________________________________________________________
    frame_lon = np.linspace(-30,720-30,frame_nr+1)
    # frame_lat = -np.tanh(np.linspace(-360,360,360)*0.2)*45
    frame_lat = -np.cos(np.linspace(-180,180,frame_nr+1)*np.pi/180)*45
    
    #___________________________________________________________________________________
    x,y,z = tpv.grid_cart3d(frame_lon[0],frame_lat[0], R_earth*10/2)
    plt.camera_position = [ np.array([x,y,z]),(0,0,0),(0,0,1)] 
    plt.show(auto_close=False, interactive_update=True, window_size=frame_wsize)
    for iframe in range(0,frame_nr):
            x,y,z = tpv.grid_cart3d(frame_lon[iframe], frame_lat[iframe], R_earth*10/2, is_deg=True)
            plt.camera_position = [ np.array([x,y,z]),(0,0,0),(0,0,1)] 
            plt.update()
#             plt.show(auto_close=False, interactive_update=True, window_size=frame_wsize)
            plt.write_frame()
    plt.close()
    
#_______________________________________________________________________________________
# do save as png
elif do_mode =='print':   
    print(' >-))))°> ')
    fname = 'image_3dglobe_'+do_var+'.png'
    x,y,z = tpv.grid_cart3d(init_lon, init_lat, R_earth*10/1.8, is_deg=True)
    plt.camera_position = [ np.array([x,y,z]),(0,0,0),(0,0,1)]  
    plt.screenshot(filename=fname, window_size=do_wsize,transparent_background=True)
    plt.show()
    

0 6000 3000
(25,)
--> draw ocean mesh
--> draw land mesh
--> draw coast lines
--> draw grid lines 
 >-))))°> 


NameError: name 'R_earth' is not defined